In [1]:
import os 
import sys
base_dir = os.path.abspath('')
curr_dir = os.path.dirname(base_dir)
if not curr_dir in sys.path: sys.path.append(curr_dir)

In [2]:
import json

from chains.base import CommonChain
from chains.self_retrival import ProgramChain
from embedding import initialize_embeddings
from models import initialize_llm
from router import PromptRouter
from utils import to_markdown

In [3]:
llm = initialize_llm("gpt4")
embeddings = initialize_embeddings()

In [4]:
chain_info = json.load(open("../chains/chain_info.json"))
chain_info

[{'name': 'financing_a_stevens_education',
  'description': 'This contains all the information in regards to helping students finance their education at Stevens by providing information regarding financing sources, institution financing programs, financial aid and state financing'},
 {'name': 'tuition_fees_and_other_expenses_for_undergraduate_students',
  'description': 'This contains all the information regarding tution fees and additional fees for undergraduate students'},
 {'name': 'tuition_fees_and_other_expenses_for_graduate_students',
  'description': 'This contains all the information regarding tution fees and additional fees for graduate students'},
 {'name': 'student_life_at_stevens',
  'description': 'This contains all the information regarding student services including sports, code of conduct and student government bodies'},
 {'name': 'student_services_at_stevens',
  'description': 'This contains all the information regarding academic tutoring, career guidance, counseling, 

In [5]:
finance_chain = CommonChain("financing_a_stevens_education", embeddings)
graudate_chain = CommonChain("graduate_education_at_stevens", embeddings)
undergraduate_chain = CommonChain("undergraduate_education_at_stevens", embeddings)
student_life = CommonChain("student_life_at_stevens", embeddings)
student_services = CommonChain("student_services_at_stevens", embeddings)
graudate_tution_chain = CommonChain("tuition_fees_and_other_expenses_for_graduate_students", embeddings)
undergraudate_tution_chain = CommonChain("tuition_fees_and_other_expenses_for_undergraduate_students", embeddings)
department_chain = ProgramChain("department_info", embeddings)

chains = {
    "financing_a_stevens_education": finance_chain,
    "tuition_fees_and_other_expenses_for_undergraduate_students": undergraudate_tution_chain,
    "tuition_fees_and_other_expenses_for_graduate_students": graudate_tution_chain,
    "student_life_at_stevens": student_life,
    "student_services_at_stevens": student_services,
    "graduate_education_at_stevens": graudate_chain,
    "undergraduate_education_at_stevens": undergraduate_chain,
    "program_info_at_stevens": department_chain,
}

In [8]:
student_life.vectorstore._collection.count()

0

In [8]:
router = PromptRouter(llm=llm, chain_info=chain_info)

In [9]:
prompt = "What are the elective courses for MS in Machine Learning?"

In [10]:
chain_name = router.get_chain_name(prompt=prompt)
print(chain_name)

program_info_at_stevens


In [11]:
curr_chain = chains.get(chain_name)
response = curr_chain.process_prompt(llm, prompt=prompt)

In [12]:
to_markdown(response["answer"])

> I don't know.